In [1]:
from pathlib import Path
import numpy as np
import pandas as pd

INDIR = Path("../socialfx&autoeq/outputs")

def load_freqs(path: Path) -> np.ndarray:
    df = pd.read_csv(path)
    return df["fc_hz"].to_numpy(float)

social_stats = pd.read_csv(INDIR / "social_stats_40.csv")
final_freqs = load_freqs(INDIR / "freqs_final.csv")

# knee (если файл есть — подхватим автоматически самый первый freqs_knee_*.csv)
knee_files = sorted(INDIR.glob("freqs_knee_*.csv"))
freqs_knee = load_freqs(knee_files[0]) if knee_files else None
knee_label = knee_files[0].stem if knee_files else None

# sweep (если есть)
df_sweep = pd.read_csv(INDIR / "sweep_results.csv") if (INDIR / "sweep_results.csv").exists() else None

social_stats.head(), final_freqs, knee_label


(   fc_hz  n_total  n_boost  n_cut   p_boost     p_cut  mean_boost  mean_cut
 0   20.0     1595      844    751  0.529154  0.470846    1.169733 -1.057404
 1   50.0     1595      858    737  0.537931  0.462069    1.240518 -1.139326
 2   83.0     1595      882    713  0.552978  0.447022    1.271045 -1.216854
 3  120.0     1595      884    711  0.554232  0.445768    1.257726 -1.188433
 4  161.0     1595      883    712  0.553605  0.446395    1.232883 -1.135992,
 array([   20.      ,    50.      ,    83.      ,   120.      ,
          159.510295,   200.043762,   254.048212,   308.562725,
          383.      ,   443.863998,   622.04351 ,   798.067155,
         1000.      ,  1485.982561,  1875.      ,  2368.080868,
         3389.648225,  4365.363477,  6934.260773,  8568.995166,
        12000.      , 14000.      , 16000.      ]),
 None)

## Количественное сравнение 40-полосного SocialFX EQ и компактного эквалайзера

### Методика сравнения

Для количественной оценки потерь информации при переходе от исходной 40-полосной частотной сетки SocialFX
к компактной архитектуре параметрического эквалайзера используется следующая процедура.

Исходная статистика направлений эквализации на 40-полосной сетке обозначается как $y_{40}$.
Проекция на компактную сетку частот и последующая реконструкция описываются цепочкой:

$y_{40} \;\rightarrow\; y_C \;\rightarrow\; \hat{y}_{40}$

где:
- $y_C$ — представление статистики на компактной сетке частот;
- $\hat{y}_{40}$ — реконструкция статистики на исходной 40-полосной сетке.

Процедура сравнения включает следующие шаги:

1. На фиксированной 40-полосной сетке SocialFX рассчитывается статистика направлений эквализации:
   - вероятность усиления $p_{boost}(f)$;
   - вероятность ослабления $p_{cut}(f)$;
   - среднее усиление при условии усиления $\mu_{boost}(f)$;
   - среднее ослабление при условии ослабления $\mu_{cut}(f)$.

2. Полученные кривые проецируются на компактную сетку центральных частот эквалайзера
   с использованием линейной интерполяции по логарифмической шкале частоты.

3. Результат реконструируется обратно на исходную 40-полосную сетку.

4. Ошибка между исходной и реконструированной статистикой используется как мера потери информации,
   возникающей исключительно вследствие редукции частотной сетки.

Таким образом, компактный эквалайзер рассматривается как оператор редукции размерности,
а ошибка реконструкции отражает степень сохранения структуры пользовательских решений.


### Используемые метрики ошибки

Оценка качества реконструкции проводится отдельно для следующих величин:

- $p_{boost}(f)$ — отражает структуру направлений пользовательских решений (усиление против ослабления);
- $\mu_{boost}(f)$ — характеризует амплитуду усилений;
- $\mu_{cut}(f)$ — характеризует амплитуду ослаблений.

В качестве основной метрики используется средняя абсолютная ошибка (MAE),
которая обладает высокой интерпретируемостью и устойчивостью к выбросам.

Для более детального анализа частотный диапазон дополнительно разбивается
на перцептуально значимые зоны: **sub-bass**, **bass**, **mids**, **presence** и **air**.

In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# --- sanity checks ---
assert "social_stats" in globals()
assert "final_freqs" in globals()

req = ["fc_hz", "p_boost", "mean_boost", "mean_cut", "n_total"]
missing = [c for c in req if c not in social_stats.columns]
if missing:
    raise ValueError(f"social_stats missing columns: {missing}")

f40 = social_stats["fc_hz"].to_numpy(float)
fC  = np.asarray(final_freqs, float)

if np.any(f40 <= 0) or np.any(fC <= 0):
    raise ValueError("Частоты должны быть > 0 для log-интерполяции.")

# базовые веса (надёжность оценки на частоте)
w_n = social_stats["n_total"].to_numpy(float)

# опциональный вес важности: если есть df/series с именем importance_* в окружении — подхватим
# ожидаем формат: колонки ["fc_hz", "importance"] или ["fc_hz","value"] или series indexed by fc_hz
w_imp = None

def _try_get_importance_weight():
    # кандидаты по распространённым именам
    candidates = [k for k in globals().keys() if "importance" in k.lower()]
    for name in candidates:
        obj = globals()[name]
        try:
            if isinstance(obj, pd.DataFrame) and "fc_hz" in obj.columns:
                if "importance" in obj.columns:
                    tmp = obj[["fc_hz", "importance"]].copy()
                    tmp = tmp.groupby("fc_hz")["importance"].mean().reindex(f40).to_numpy(float)
                    return tmp
                if "value" in obj.columns:
                    tmp = obj[["fc_hz", "value"]].copy()
                    tmp = tmp.groupby("fc_hz")["value"].mean().reindex(f40).to_numpy(float)
                    return tmp
            if isinstance(obj, pd.Series):
                # если индекс частоты
                tmp = obj.reindex(f40).to_numpy(float)
                if np.all(np.isfinite(tmp)):
                    return tmp
        except Exception:
            pass
    return None

w_imp = _try_get_importance_weight()

# итоговый вес для WMAE: n_total * importance (если нашлась)
if w_imp is not None and np.any(np.isfinite(w_imp)):
    w = w_n * np.nan_to_num(w_imp, nan=0.0)
    weight_mode = "n_total * importance"
else:
    w = w_n
    weight_mode = "n_total"

print("Weight mode:", weight_mode)
print("40-band freqs:", f40[:5], "...", f40[-5:])
print("Compact freqs:", fC)


Weight mode: n_total
40-band freqs: [ 20.  50.  83. 120. 161.] ... [12474. 13984. 15675. 17566. 19682.]
Compact freqs: [   20.          50.          83.         120.         159.510295
   200.043762   254.048212   308.562725   383.         443.863998
   622.04351    798.067155  1000.        1485.982561  1875.
  2368.080868  3389.648225  4365.363477  6934.260773  8568.995166
 12000.       14000.       16000.      ]


In [3]:
def log_interp(x_hz, y, xq_hz):
    x = np.asarray(x_hz, float)
    y = np.asarray(y, float)
    xq = np.asarray(xq_hz, float)

    lx = np.log10(x)
    lxq = np.log10(xq)

    order = np.argsort(lx)
    lx = lx[order]
    y = y[order]

    return np.interp(lxq, lx, y)

def reproject_40_C_40(y40, kind="value"):
    """
    kind: "prob" -> клип в [0,1]
          "value" -> без клипа
    """
    y40 = np.asarray(y40, float)

    yC = log_interp(f40, y40, fC)
    y40_hat = log_interp(fC, yC, f40)

    if kind == "prob":
        yC = np.clip(yC, 0.0, 1.0)
        y40_hat = np.clip(y40_hat, 0.0, 1.0)

    return yC, y40_hat

def MAE(a, b):
    a = np.asarray(a, float); b = np.asarray(b, float)
    return float(np.mean(np.abs(a - b)))

def WMAE(a, b, w):
    a = np.asarray(a, float); b = np.asarray(b, float); w = np.asarray(w, float)
    w = np.clip(w, 0, None)
    if w.sum() == 0:
        return MAE(a, b)
    return float(np.sum(w * np.abs(a - b)) / np.sum(w))


In [4]:
ZONES = {
    "sub_bass": (20, 60),
    "bass":     (60, 200),
    "mids":     (200, 2000),
    "presence": (2000, 6000),
    "air":      (6000, 20000),
}

def mask_zone(lo, hi):
    return (f40 >= lo) & (f40 < hi)

metrics = {
    "p_boost": ("prob",  social_stats["p_boost"].to_numpy(float)),
    "mean_boost": ("value", social_stats["mean_boost"].to_numpy(float)),
    "mean_cut":   ("value", social_stats["mean_cut"].to_numpy(float)),
}

# если есть p_cut — тоже посчитаем
if "p_cut" in social_stats.columns:
    metrics["p_cut"] = ("prob", social_stats["p_cut"].to_numpy(float))

recon = {}  # metric -> dict(y40, yC, y40_hat)
rows = []

for name, (kind, y40) in metrics.items():
    yC, y40_hat = reproject_40_C_40(y40, kind=kind)
    recon[name] = dict(y40=y40, yC=yC, y40_hat=y40_hat)

    # overall
    rows.append(dict(region="overall", metric=name, mae=MAE(y40, y40_hat), wmae=WMAE(y40, y40_hat, w)))

    # zones
    for z, (lo, hi) in ZONES.items():
        m = mask_zone(lo, hi)
        if not np.any(m):
            continue
        rows.append(dict(region=z, metric=name, mae=MAE(y40[m], y40_hat[m]), wmae=WMAE(y40[m], y40_hat[m], w[m])))

df_errors = pd.DataFrame(rows).sort_values(["region","metric"]).reset_index(drop=True)
df_errors


,region,metric,mae,wmae
0,air,mean_boost,0.009959,0.009959
1,air,mean_cut,0.006295,0.006295
2,air,p_boost,0.002291,0.002291
3,air,p_cut,0.002291,0.002291
4,bass,mean_boost,0.000001,0.000001
5,bass,mean_cut,0.000135,0.000135
6,bass,p_boost,0.000015,0.000015
7,bass,p_cut,0.000015,0.000015
8,mids,mean_boost,0.009116,0.009116
9,mids,mean_cut,0.010977,0.010977


In [5]:
def top_errors(metric_name, top_k=12, weighted=True):
    rr = recon[metric_name]
    err = np.abs(rr["y40"] - rr["y40_hat"])

    out = pd.DataFrame({
        "fc_hz": f40,
        "y_true": rr["y40"],
        "y_hat": rr["y40_hat"],
        "abs_err": err,
        "n_total": w_n,
    })

    if weighted:
        out["weight"] = w
        out["w_abs_err"] = out["abs_err"] * out["weight"]
        out = out.sort_values("w_abs_err", ascending=False).head(top_k)
    else:
        out = out.sort_values("abs_err", ascending=False).head(top_k)

    return out.reset_index(drop=True)

top_pboost = top_errors("p_boost", top_k=12, weighted=True)
top_mboost = top_errors("mean_boost", top_k=12, weighted=True)
top_mcut   = top_errors("mean_cut", top_k=12, weighted=True)

top_pboost, top_mboost, top_mcut


(      fc_hz    y_true     y_hat   abs_err  n_total  weight  w_abs_err
 0    1273.0  0.447022  0.456379  0.009357   1595.0  1595.0  14.923973
 1    1114.0  0.450784  0.457643  0.006859   1595.0  1595.0  10.940791
 2    4941.0  0.497179  0.490974  0.006205   1595.0  1595.0   9.896280
 3    5556.0  0.486520  0.480591  0.005929   1595.0  1595.0   9.456625
 4   11124.0  0.442633  0.438283  0.004351   1595.0  1595.0   6.939060
 5    7875.0  0.442633  0.446922  0.004288   1595.0  1595.0   6.840068
 6    1652.0  0.465204  0.461028  0.004176   1595.0  1595.0   6.660955
 7    2126.0  0.473354  0.477507  0.004153   1595.0  1595.0   6.623328
 8    9917.0  0.433856  0.437978  0.004122   1595.0  1595.0   6.574222
 9    6244.0  0.473981  0.470259  0.003722   1595.0  1595.0   5.936772
 10   3070.0  0.487147  0.490399  0.003251   1595.0  1595.0   5.185984
 11    537.0  0.519749  0.522781  0.003032   1595.0  1595.0   4.835706,
       fc_hz    y_true     y_hat   abs_err  n_total  weight  w_abs_err
 0   

In [6]:
def plot_reconstruction(metric_name, title=None):
    rr = recon[metric_name]
    y = rr["y40"]
    yh = rr["y40_hat"]
    err = np.abs(y - yh)

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=f40, y=y, mode="lines+markers", name="orig (SocialFX 40)", marker=dict(size=6)))
    fig.add_trace(go.Scatter(x=f40, y=yh, mode="lines+markers", name="recon (40→C→40)", marker=dict(size=6)))

    fig.update_xaxes(type="log", title="Frequency (Hz, log)")
    fig.update_yaxes(title=metric_name)
    fig.update_layout(title=title or f"Reconstruction on 40-band grid: {metric_name}", legend=dict(orientation="h"))

    # отдельная фигура для ошибки
    fig_err = go.Figure()
    fig_err.add_trace(go.Bar(x=f40, y=err, name="|error|"))
    fig_err.update_xaxes(type="log", title="Frequency (Hz, log)")
    fig_err.update_yaxes(title="Absolute error")
    fig_err.update_layout(title=f"Absolute error on 40-band grid: {metric_name}")

    return fig, fig_err

fig1, fig1e = plot_reconstruction("p_boost", "p_boost: orig vs recon (40→compact→40)")
fig2, fig2e = plot_reconstruction("mean_boost", "mean_boost: orig vs recon (40→compact→40)")
fig3, fig3e = plot_reconstruction("mean_cut", "mean_cut: orig vs recon (40→compact→40)")

fig1.show(); fig1e.show()
fig2.show(); fig2e.show()
fig3.show(); fig3e.show()


In [7]:
pivot = df_errors.pivot(index="region", columns="metric", values="wmae").copy()
pivot = pivot.loc[["overall","sub_bass","bass","mids","presence","air"]].dropna(how="all")
pivot


metric,mean_boost,mean_cut,p_boost,p_cut
region,,,,
overall,0.009828,0.010244,0.002014,0.002014
sub_bass,0.000000,0.000000,0.000000,0.000000
bass,0.000001,0.000135,0.000015,0.000015
mids,0.009116,0.010977,0.002121,0.002121
presence,0.016314,0.019496,0.002609,0.002609
air,0.009959,0.006295,0.002291,0.002291


### Анализ результатов по частотным зонам

**Низкие частоты (sub-bass, bass)**
В диапазоне до ~200 Гц ошибка реконструкции практически отсутствует для всех метрик. Это указывает на то, что низкочастотные предпочтения пользователей имеют гладкую структуру и хорошо аппроксимируются малым числом полос.

**Средние частоты (mids)**
В диапазоне 200–2000 Гц наблюдается умеренный рост ошибки, главным образом по амплитудным метрикам. При этом структура направлений эквализации $( p_{boost} )$ сохраняется с высокой точностью, что указывает на корректное воспроизведение общей тенденции пользовательских решений.

**Область presence (≈2–6 кГц)**
Наибольшая ошибка реконструкции наблюдается в области presence, особенно для $( \mu_{boost} )$ и $( \mu_{cut} )$. Это подтверждает высокую вариативность пользовательских предпочтений в данном диапазоне и указывает на принципиальное ограничение компактных архитектур эквалайзеров в этой зоне.

**Высокие частоты и air**
В верхнем диапазоне ошибка остаётся умеренной и носит локальный характер. Это указывает на возможность адекватного описания высокочастотной коррекции с помощью ограниченного числа полос или функциональных форм (например, high-shelf).

### Анализ локальных ошибок

Анализ частот с максимальной абсолютной ошибкой показывает, что наибольшие расхождения между исходной и реконструированной статистикой концентрируются:
- в области presence (≈1–6 кГц);
- в отдельных точках верхнего диапазона при реконструкции амплитуд усилений и ослаблений.

При этом даже в этих зонах структура направлений эквализации сохраняется с высокой точностью.

### Выводы

1. Компактный параметрический эквалайзер с существенно меньшим числом полос способен воспроизводить статистику пользовательских решений SocialFX с малой средней ошибкой.
2. Направления эквализации (boost vs cut) сохраняются значительно лучше, чем амплитудные характеристики.
3. Основные потери информации сосредоточены в области presence, что отражает фундаментальную субъективную вариативность этого диапазона.
4. Низкие и высокие частоты допускают агрессивную редукцию числа полос без заметной потери статистической информации.
5. Полученные результаты подтверждают корректность выбранной архитектуры компактного эквалайзера и позволяют количественно обосновать компромисс между сложностью модели и точностью воспроизведения пользовательских предпочтений.

### Ограничения метода

Следует отметить, что данное сравнение оценивает исключительно сохранение статистики пользовательских решений и не является прямой оценкой субъективного качества звучания. Тем не менее, такой подход обеспечивает методологически чистую и воспроизводимую основу для анализа архитектур параметрических эквалайзеров.

## Калибровка компактного эквалайзера по ошибке реконструкции

### Идея калибровки

Для уменьшения систематической ошибки реконструкции без увеличения числа полос
вводится калибровка компактного представления в виде добавки на компактной сетке:

$y_C^{cal} = y_C + \Delta_C$

где $\Delta_C$ подбирается таким образом, чтобы минимизировать ошибку
между исходной статистикой $y_{40}$ и реконструкцией $\hat{y}_{40}$.

Калибровка формулируется как задача взвешенного наименьшего квадрата (WLS)
с L2-регуляризацией:

$\min_{\Delta_C} \; \| W ( y_{40} - \hat{y}_{40} ) \|_2^2 + \lambda \| \Delta_C \|_2^2$

где:
- $W$ — диагональная матрица весов (в эксперименте используется число наблюдений $n_{total}$);
- $\lambda$ — коэффициент регуляризации, ограничивающий избыточную вариативность $\Delta_C$.

Таким образом, калибровка компенсирует систематические искажения,
вызванные редукцией сетки частот, сохраняя компактность и интерпретируемость эквалайзера.


In [8]:
import numpy as np
import pandas as pd

# --- матрица интерполяции по log f: y_out = A @ y_in
def interp_matrix_log(x_in, x_out):
    x_in = np.asarray(x_in, float)
    x_out = np.asarray(x_out, float)
    lin = np.log10(x_in)
    lout = np.log10(x_out)

    order = np.argsort(lin)
    lin = lin[order]
    inv_order = np.argsort(order)

    A = np.zeros((len(x_out), len(x_in)), float)

    for i, t in enumerate(lout):
        if t <= lin[0]:
            A[i, 0] = 1.0
            continue
        if t >= lin[-1]:
            A[i, -1] = 1.0
            continue
        j = np.searchsorted(lin, t) - 1
        j = np.clip(j, 0, len(lin) - 2)
        x0, x1 = lin[j], lin[j+1]
        w1 = (t - x0) / (x1 - x0)
        w0 = 1.0 - w1
        A[i, j]   = w0
        A[i, j+1] = w1

    # вернуть к исходному порядку x_in
    A = A[:, inv_order]
    return A

f40 = social_stats["fc_hz"].to_numpy(float)
fC  = np.asarray(final_freqs, float)

A_40_to_C = interp_matrix_log(f40, fC)   # yC = A_40_to_C @ y40
A_C_to_40 = interp_matrix_log(fC, f40)   # y40_hat = A_C_to_40 @ yC

w = social_stats["n_total"].to_numpy(float)
W = np.diag(np.sqrt(np.clip(w, 1e-12, None)))  # sqrt для WLS


In [9]:
def calibrate_delta_on_compact(y40, kind="value", reg=1e-3):
    """
    Ищем delta_C: минимизируем || W ( y40 - A_C_to_40 (A_40_to_C y40 + delta_C) ) ||^2 + reg||delta||^2
    """
    y40 = np.asarray(y40, float)

    # исходная проекция 40 -> C
    yC0 = A_40_to_C @ y40

    # целевая поправка на 40-сетке
    resid_40 = y40 - (A_C_to_40 @ yC0)

    # хотим: A_C_to_40 @ delta_C ≈ resid_40
    # решаем WLS + L2-регуляризация
    B = A_C_to_40
    Bw = W @ B
    rw = W @ resid_40

    # (B^T W^2 B + reg I) delta = B^T W^2 r
    lhs = Bw.T @ Bw + reg * np.eye(B.shape[1])
    rhs = Bw.T @ rw
    delta_C = np.linalg.solve(lhs, rhs)

    yC_cal = yC0 + delta_C
    y40_cal = A_C_to_40 @ yC_cal

    if kind == "prob":
        yC_cal = np.clip(yC_cal, 0, 1)
        y40_cal = np.clip(y40_cal, 0, 1)

    return dict(yC0=yC0, delta_C=delta_C, yC_cal=yC_cal, y40_cal=y40_cal)


In [10]:
def MAE(a,b):
    return float(np.mean(np.abs(np.asarray(a)-np.asarray(b))))

def report_before_after(metric_name, kind):
    y40 = social_stats[metric_name].to_numpy(float)

    # до калибровки
    yC0 = A_40_to_C @ y40
    y40_hat = A_C_to_40 @ yC0
    if kind == "prob":
        y40_hat = np.clip(y40_hat, 0, 1)

    # после калибровки
    cal = calibrate_delta_on_compact(y40, kind=kind, reg=1e-3)
    y40_cal = cal["y40_cal"]

    print(metric_name)
    print("  MAE before:", MAE(y40, y40_hat))
    print("  MAE after :", MAE(y40, y40_cal))

# пробуем на амплитудах (там больше ошибка)
report_before_after("mean_boost", "value")
report_before_after("mean_cut", "value")
report_before_after("p_boost", "prob")


mean_boost
  MAE before: 0.009827781212998826
  MAE after : 0.009197377909576315
mean_cut
  MAE before: 0.010244376423317733
  MAE after : 0.008785911927929806
p_boost
  MAE before: 0.002013526694464825
  MAE after : 0.0016521686768321401



### Интерпретация результатов калибровки

1. Наибольшее относительное улучшение достигается для $p_{boost}(f)$,
   что указывает на более точное воспроизведение структуры направлений пользовательских решений.

2. Существенное снижение ошибки для $\mu_{cut}(f)$ показывает,
   что калибровка эффективно компенсирует искажения амплитуды ослаблений,
   возникающие при редукции частотной сетки.

3. Улучшение достигается без увеличения числа полос эквалайзера,
   что подтверждает эффективность калибровки как метода посткоррекции компактной архитектуры.

### Выводы

1. Компактный параметрический эквалайзер с существенно меньшим числом полос
   способен воспроизводить статистику пользовательских решений SocialFX
   с малой средней ошибкой.

2. Направления эквализации (boost против cut) сохраняются значительно лучше,
   чем амплитудные характеристики.

3. Основные потери информации сосредоточены в области presence,
   что отражает высокую субъективную вариативность данного диапазона.

4. Низкие и высокие частоты допускают агрессивную редукцию числа полос
   без заметной потери статистической информации.

5. Калибровка компактного эквалайзера по ошибке реконструкции
   позволяет дополнительно снизить потери информации,
   не усложняя архитектуру модели.


### Ограничения метода

Следует отметить, что предложенный метод оценивает исключительно
сохранение статистики пользовательских решений
и не является прямой оценкой субъективного качества звучания.
Тем не менее, он обеспечивает методологически чистую,
воспроизводимую и количественно обоснованную основу
для анализа архитектур параметрических эквалайзеров.

## Влияние редукции эквалайзера на сложность модели и вычислительные затраты

Переход от 40-полосной архитектуры к компактному эквалайзеру
приводит к снижению числа параметров примерно на 40–45%.
В терминах цифровой обработки сигналов это означает уменьшение числа
бикуад-фильтров, обрабатывающих сигнал в реальном времени.

Поскольку вычислительная сложность параметрического эквалайзера
линейно зависит от числа полос, редукция архитектуры приводит
к пропорциональному снижению вычислительной нагрузки.

Полученное снижение сложности достигается
без существенной потери точности воспроизведения статистики
пользовательских решений, что подтверждается малой ошибкой реконструкции
и дополнительным снижением ошибки после калибровки компактной модели.

Таким образом, предложенный подход обеспечивает выгодный компромисс
между точностью, интерпретируемостью и вычислительной эффективностью
параметрического эквалайзера.

## Выбор числа полос и анализ компромисса «точность–сложность»

Целью данного этапа является количественная оценка того, насколько сильно можно сократить
число полос параметрического эквалайзера, сохранив при этом статистическое поведение
40-полосного эквалайзера SocialFX.

В качестве эталона используется 40-полосная статистика SocialFX, описанная следующими метриками:
- вероятность применения усиления (p_boost),
- среднее значение усиления (mean_boost),
- среднее значение ослабления (mean_cut).

Для каждой компактной конфигурации эквалайзера с числом полос N выполняется следующая процедура:

1. Проекция 40-полосной статистики на компактную сетку частот:

   $$
   y_{40} \;\xrightarrow{\,40 \to C\,}\; y_C
   $$

2. Обратная реконструкция на 40-полосную сетку:

   $$
   y_C \;\xrightarrow{\,C \to 40\,}\; \hat{y}_{40}
   $$

3. Вычисление ошибки между исходной и реконструированной статистикой
   с использованием MAE и WMAE (с весами по количеству наблюдений).


In [11]:
import numpy as np
import pandas as pd

# Требуемые переменные из твоего ноутбука:
assert "social_stats" in globals()
assert "final_freqs" in globals()

# Якоря: если ANCHORS уже есть — используем, если нет — восстановим из твоей логики
if "ANCHORS" not in globals():
    ANCHORS = [1000.0, 12000.0, 14000.0, 16000.0]

f40 = social_stats["fc_hz"].to_numpy(float)

# Кандидатные частоты для отбора.
# Самый честный вариант: кандидаты = 40 частот SocialFX + якоря
cand = np.unique(np.concatenate([f40, np.asarray(ANCHORS, float)]))
cand = np.sort(cand)

# Данные, которые сравниваем (те же, что и раньше)
y_pboost = social_stats["p_boost"].to_numpy(float)
y_mboost = social_stats["mean_boost"].to_numpy(float)
y_mcut   = social_stats["mean_cut"].to_numpy(float)

# веса (у тебя n_total одинаковый, но оставим общо)
w = social_stats["n_total"].to_numpy(float)
w = np.clip(w, 1e-12, None)

print("Candidates:", len(cand), "freqs")
print("Anchors:", ANCHORS)


Candidates: 44 freqs
Anchors: [1000.0, 12000.0, 14000.0, 16000.0]


In [12]:
def interp_matrix_log(x_in, x_out):
    x_in = np.asarray(x_in, float)
    x_out = np.asarray(x_out, float)
    lin = np.log10(x_in)
    lout = np.log10(x_out)

    order = np.argsort(lin)
    lin_sorted = lin[order]
    inv_order = np.argsort(order)

    A = np.zeros((len(x_out), len(x_in)), float)

    for i, t in enumerate(lout):
        if t <= lin_sorted[0]:
            A[i, 0] = 1.0
            continue
        if t >= lin_sorted[-1]:
            A[i, -1] = 1.0
            continue
        j = np.searchsorted(lin_sorted, t) - 1
        j = np.clip(j, 0, len(lin_sorted) - 2)
        x0, x1 = lin_sorted[j], lin_sorted[j+1]
        w1 = (t - x0) / (x1 - x0)
        w0 = 1.0 - w1
        A[i, j]   = w0
        A[i, j+1] = w1

    A = A[:, inv_order]
    return A

def MAE(a, b):
    a = np.asarray(a, float); b = np.asarray(b, float)
    return float(np.mean(np.abs(a - b)))

def WMAE(a, b, w):
    a = np.asarray(a, float); b = np.asarray(b, float); w = np.asarray(w, float)
    return float(np.sum(w * np.abs(a - b)) / np.sum(w))

def project_reconstruct_error(fC, y40, kind="value"):
    """
    Возвращает (mae, wmae) для реконструкции 40→C→40
    """
    A_40_C = interp_matrix_log(f40, fC)
    A_C_40 = interp_matrix_log(fC, f40)

    yC = A_40_C @ y40
    y40_hat = A_C_40 @ yC

    if kind == "prob":
        y40_hat = np.clip(y40_hat, 0, 1)

    return MAE(y40, y40_hat), WMAE(y40, y40_hat, w)


In [13]:
def score_model(fC, weights=(0.45, 0.35, 0.20)):
    """
    Возвращает dict с ошибками и итоговым score.
    weights = (w_cut, w_boost, w_p)
    """
    w_cut, w_boost, w_p = weights

    mae_p,  wmae_p  = project_reconstruct_error(fC, y_pboost, kind="prob")
    mae_mb, wmae_mb = project_reconstruct_error(fC, y_mboost, kind="value")
    mae_mc, wmae_mc = project_reconstruct_error(fC, y_mcut, kind="value")

    # итоговый скор — взвешенный WMAE (чтобы быть устойчивым)
    score = w_cut*wmae_mc + w_boost*wmae_mb + w_p*wmae_p

    return {
        "score": score,
        "wmae_p_boost": wmae_p,
        "wmae_mean_boost": wmae_mb,
        "wmae_mean_cut": wmae_mc,
        "mae_p_boost": mae_p,
        "mae_mean_boost": mae_mb,
        "mae_mean_cut": mae_mc,
        "N": len(fC),
    }


In [14]:
def greedy_select_freqs(
    cand_hz,
    anchors_hz,
    N_target,
    must_include_ends=True,
    score_weights=(0.45, 0.35, 0.20),
):
    cand_hz = np.asarray(cand_hz, float)
    anchors_hz = np.asarray(anchors_hz, float)

    base = list(anchors_hz)

    if must_include_ends:
        base += [float(np.min(f40)), float(np.max(f40))]

    # уникализируем и сортируем
    selected = np.unique(np.array(base, float))
    selected = np.sort(selected)

    # кандидаты, которые ещё можно добавить
    remaining = np.array([c for c in cand_hz if c not in set(selected)], float)

    history = []
    # посчитаем стартовый скор
    best = score_model(selected, weights=score_weights)
    history.append({"step": len(selected), "added": None, **best})

    while len(selected) < N_target and remaining.size > 0:
        best_candidate = None
        best_result = None

        for c in remaining:
            trial = np.sort(np.unique(np.append(selected, c)))
            res = score_model(trial, weights=score_weights)
            if (best_result is None) or (res["score"] < best_result["score"]):
                best_result = res
                best_candidate = c

        selected = np.sort(np.unique(np.append(selected, best_candidate)))
        remaining = np.array([c for c in remaining if c != best_candidate], float)

        history.append({"step": len(selected), "added": float(best_candidate), **best_result})

    return selected, pd.DataFrame(history)

# Пример: попробуем разные N, например 8..26


In [15]:
def knee_point(df, x="N", y="score"):
    """
    Простая knee-эвристика: максимальное расстояние до прямой между концами кривой.
    """
    d = df.sort_values(x).copy()
    X = d[x].to_numpy(float)
    Y = d[y].to_numpy(float)

    # нормировка
    Xn = (X - X.min()) / (X.max() - X.min() + 1e-12)
    Yn = (Y - Y.min()) / (Y.max() - Y.min() + 1e-12)

    # линия между концами
    x1, y1 = Xn[0], Yn[0]
    x2, y2 = Xn[-1], Yn[-1]
    # расстояние точки до линии
    num = np.abs((y2 - y1)*Xn - (x2 - x1)*Yn + x2*y1 - y2*x1)
    den = np.sqrt((y2 - y1)**2 + (x2 - x1)**2) + 1e-12
    dist = num / den

    idx = int(np.argmax(dist))
    return int(d.iloc[idx][x]), float(dist[idx])

def sweep_N(
    N_list,
    cand_hz=cand,
    anchors_hz=ANCHORS,
    score_weights=(0.45, 0.35, 0.20),
):
    results = []
    models = {}

    for N in N_list:
        fC_sel, hist = greedy_select_freqs(
            cand_hz=cand_hz,
            anchors_hz=anchors_hz,
            N_target=int(N),
            must_include_ends=True,
            score_weights=score_weights,
        )
        res = score_model(fC_sel, weights=score_weights)
        results.append(res)
        models[int(N)] = {"freqs": fC_sel, "history": hist}

    df = pd.DataFrame(results).sort_values("N").reset_index(drop=True)
    kN, kdist = knee_point(df, x="N", y="score")
    return df, models, (kN, kdist)

N_list = list(range(8, 27))  # можно расширить
df_sweep, models_by_N, knee = sweep_N(N_list)

df_sweep, knee


(       score  wmae_p_boost  wmae_mean_boost  wmae_mean_cut  mae_p_boost  \
 0   0.031103      0.012690         0.035404       0.035941     0.012690   
 1   0.025671      0.009290         0.032355       0.027753     0.009290   
 2   0.018530      0.006772         0.020663       0.022097     0.006772   
 3   0.015574      0.006915         0.017341       0.018047     0.006915   
 4   0.014365      0.006446         0.016329       0.016356     0.006446   
 5   0.012523      0.005576         0.014195       0.014310     0.005576   
 6   0.011339      0.004460         0.013707       0.012554     0.004460   
 7   0.010322      0.004425         0.012416       0.011315     0.004425   
 8   0.009577      0.004441         0.010382       0.011233     0.004441   
 9   0.007893      0.004162         0.009393       0.008385     0.004162   
 10  0.007028      0.004033         0.008994       0.006830     0.004033   
 11  0.006532      0.003842         0.008089       0.006517     0.003842   
 12  0.00606

### Sweep по числу полос

Был выполнен перебор конфигураций компактного эквалайзера с числом полос
в диапазоне $N \in [8, 26]$.
Для каждой конфигурации вычислялась агрегированная ошибка `score`,
объединяющая ошибки по всем метрикам.

Результаты показывают монотонное уменьшение ошибки при увеличении числа полос,
однако скорость улучшения резко снижается после определённого значения N.

In [16]:
kN, kdist = knee
print("Knee (recommended N):", kN, "knee_strength:", kdist)

best_freqs = models_by_N[kN]["freqs"]
print("Recommended freqs:", best_freqs)
print("Count:", len(best_freqs))


Knee (recommended N): 11 knee_strength: 0.2749671803995561
Recommended freqs: [   20.   120.   259.   628.  1000.  4392.  4941. 12000. 14000. 16000.
 19682.]
Count: 11


### Анализ knee-point

Для формального выбора оптимального числа полос был применён метод *knee-point detection*
к зависимости `score(N)`.

В результате был обнаружен выраженный излом кривой при:

- **N = 11**
- **knee_strength ≈ 0.275**

Это означает, что:
- при $N < 11$ добавление каждой новой полосы существенно снижает ошибку,
- при $N > 11$ наблюдается эффект убывающей отдачи (diminishing returns).

### Сравнение выбранных конфигураций

Для дальнейшего анализа были выбраны две конфигурации:

- **Final freqs (N = 23)** — компактный эквалайзер с минимальной потерей точности;
- **Knee configuration (N = 11)** — максимально компактная конфигурация,
  обеспечивающая приемлемую аппроксимацию.

In [17]:
res_current = score_model(np.asarray(final_freqs, float))
res_knee    = score_model(best_freqs)

pd.DataFrame([
    {"model": "current_final_freqs", **res_current},
    {"model": f"knee_N_{kN}", **res_knee},
])[["model","N","score","wmae_p_boost","wmae_mean_boost","wmae_mean_cut","mae_p_boost","mae_mean_boost","mae_mean_cut"]]


,model,N,score,wmae_p_boost,wmae_mean_boost,wmae_mean_cut,mae_p_boost,mae_mean_boost,mae_mean_cut
0,current_final_freqs,23,0.008452,0.002014,0.009828,0.010244,0.002014,0.009828,0.010244
1,knee_N_11,11,0.015574,0.006915,0.017341,0.018047,0.006915,0.017341,0.018047


При уменьшении числа полос более чем в 2 раза
ошибка увеличивается примерно в 1.8 раза,
что указывает на благоприятный компромисс между точностью и сложностью модели.

### Интерпретация полученных частот

Рекомендуемая knee-конфигурация (N = 11) содержит полосы,
естественным образом соответствующие основным частотным зонам:

- sub-bass и bass (20–120 Гц),
- low-mids и mids (250–1000 Гц),
- presence (4–5 кГц),
- air (12–16 кГц).

Это подтверждает, что метод выбора полос
не только минимизирует ошибку,
но и приводит к интерпретируемой и практически применимой архитектуре эквалайзера.

In [18]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

ZONES = {
    "sub_bass": (20, 60),
    "bass":     (60, 200),
    "mids":     (200, 2000),
    "presence": (2000, 6000),
    "air":      (6000, 20000),
}

def band_zone(freq_hz):
    f = float(freq_hz)
    for name, (lo, hi) in ZONES.items():
        if lo <= f < hi:
            return name
    return "out_of_range"

def describe_bands(freqs):
    freqs = np.asarray(freqs, float)
    return pd.DataFrame({
        "band_idx": np.arange(1, len(freqs) + 1),
        "fc_hz": freqs,
        "zone": [band_zone(f) for f in freqs],
    })


In [19]:
def recon_via_freqs(metric_name, freqs_C, kind=None):
    """
    Возвращает (y40, y40_hat) для заданной компактной сетки freqs_C.
    Требует: f40, social_stats, log_interp (у тебя уже есть).
    """
    y40 = social_stats[metric_name].to_numpy(float)

    if kind is None:
        kind = "prob" if metric_name in ("p_boost", "p_cut") else "value"

    yC = log_interp(f40, y40, freqs_C)
    y40_hat = log_interp(freqs_C, yC, f40)

    if kind == "prob":
        y40_hat = np.clip(y40_hat, 0.0, 1.0)

    return y40, y40_hat


In [20]:
def plot_reconstruction_compare(
    metric_name,
    freqs_A, label_A,
    freqs_B, label_B,
    title=None,
):
    kind = "prob" if metric_name in ("p_boost", "p_cut") else "value"

    y, yA = recon_via_freqs(metric_name, freqs_A, kind=kind)
    _, yB = recon_via_freqs(metric_name, freqs_B, kind=kind)

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=f40, y=y, mode="lines+markers",
        name="orig (SocialFX 40)", marker=dict(size=6)
    ))
    fig.add_trace(go.Scatter(
        x=f40, y=yA, mode="lines+markers",
        name=f"A: {label_A} (N={len(freqs_A)})", marker=dict(size=6)
    ))
    fig.add_trace(go.Scatter(
        x=f40, y=yB, mode="lines+markers",
        name=f"B: {label_B} (N={len(freqs_B)})", marker=dict(size=6)
    ))

    fig.update_xaxes(type="log", title="Frequency (Hz, log)")
    fig.update_yaxes(title=metric_name)
    fig.update_layout(
        title=title or f"{metric_name}: orig vs A vs B",
        legend=dict(orientation="h")
    )
    return fig


In [21]:
# A = твой текущий эквалайзер
freqs_A = np.asarray(final_freqs, float)
label_A = "final_freqs"

# B = knee
kN = int(knee[0]) if "knee" in globals() else None
if kN is None:
    raise ValueError("Не найден knee: переменная knee должна быть (kN, ...)")

freqs_B = np.asarray(models_by_N[kN]["freqs"], float)
label_B = f"knee (N={kN})"

# Графики
fig1 = plot_reconstruction_compare("p_boost", freqs_A, label_A, freqs_B, label_B,
                                   title="p_boost: final_freqs vs knee")
fig2 = plot_reconstruction_compare("mean_boost", freqs_A, label_A, freqs_B, label_B,
                                   title="mean_boost: final_freqs vs knee")
fig3 = plot_reconstruction_compare("mean_cut", freqs_A, label_A, freqs_B, label_B,
                                   title="mean_cut: final_freqs vs knee")

fig1.show(); fig2.show(); fig3.show()

# Таблицы "полоса → зона"
display(describe_bands(freqs_A))
display(describe_bands(freqs_B))


,band_idx,fc_hz,zone
0,1,20.000000,sub_bass
1,2,50.000000,sub_bass
2,3,83.000000,bass
3,4,120.000000,bass
4,5,159.510295,bass
5,6,200.043762,mids
6,7,254.048212,mids
7,8,308.562725,mids
8,9,383.000000,mids
9,10,443.863998,mids


,band_idx,fc_hz,zone
0,1,20.0,sub_bass
1,2,120.0,bass
2,3,259.0,mids
3,4,628.0,mids
4,5,1000.0,mids
5,6,4392.0,presence
6,7,4941.0,presence
7,8,12000.0,air
8,9,14000.0,air
9,10,16000.0,air



### Вывод

Проведённый анализ показывает, что 40-полосный эквалайзер SocialFX
обладает значительной избыточностью.
Его статистическое поведение может быть воспроизведено:

- почти без потерь — с использованием ~23 полос;
- с приемлемой потерей точности — с использованием ~11 полос.

Таким образом, компактный параметрический эквалайзер
может быть существенно упрощён
без критической деградации описываемых пользовательских решений,
что открывает возможности для более лёгких, быстрых
и интерпретируемых моделей эквализации.